Importando UsersID da planilha

In [ ]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df ['UserID'].tolist()
print(user_ids)

Obtendo dados da API santander.

In [ ]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

##Adicionei essa condicao para filtrar os clientes que podem ser investidores em potencial , que possuem mais de 0.0 na conta.
##Caso o cliente esteja com a conta zerada ou negativa, ele nao receberá uma mensagem.
filtered_users = [user for user in users if user.get('account').get('balance', 0) > 0.0 ]



print(json.dumps(filtered_users, indent=2))     

Interação com a Openai para gerar mensagens personalizadas.

In [ ]:
import openai
openai_api_key = 'Minha chave API'

openai.api_key = openai_api_key

def generate_ai_news(user):
 completion = openai.ChatCompletion.create(
   model="gpt-3.5-turbo-16k",
   messages=[
    {
        "role": "system", "content": "Você é um especialista em marketing bancário."
    },
    {
        "role": "user", "content": f"Crie uma mensagem para {user[ 'name' ]} sobre a importancia de investir 20% do capital. (máximo de 100 caracteres)"
    }
  ]
 )
 return completion.choices[0].message.content.strip('\"')

#Enviar a mensagem personalizada para os clientes com potencial de investimento.
for user in filtered_users:
  news = generate_ai_news(user)
  print(news)